In [1]:
import os
os.chdir("../")
os.getcwd()

'/home/izam/coding/Customer-Churn'

In [2]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    params: dict
    target_col: str
    permanent_path: str
    auto_select: bool

In [3]:
from CustomerChurn.constants import *
from CustomerChurn.utils.common import read_yaml, create_directories

In [4]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        target_col = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path = config.train_data_path,
            test_data_path = config.test_data_path,
            model_name = config.model_name,
            params = self.params,
            target_col=target_col.name,
            permanent_path=config.permanent_path,
            auto_select=config.auto_select,
        )

        return model_trainer_config

In [1]:
import pandas as pd
from box import ConfigBox
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score
from CustomerChurn import logger
from CustomerChurn.utils.common import save_bin, save_bin_dup

# models
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from xgboost import XGBClassifier
from scipy.stats import reciprocal, uniform

In [6]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config
        train = pd.read_csv(self.config.train_data_path)
        test = pd.read_csv(self.config.test_data_path)

        self.X_train= train.drop([self.config.target_col], axis=1)
        self.y_train = train[self.config.target_col]
        self.X_test = test.drop([self.config.target_col], axis=1)
        self.y_test = test[self.config.target_col]   


    def _randomized_search(self, name,clf,params, runs=50): 
        rand_clf = RandomizedSearchCV(clf, params, n_iter=runs, cv=5, n_jobs=4, random_state=2, verbose=0)     

        rand_clf.fit(self.X_train, self.y_train) 
        best_model = rand_clf.best_estimator_
        
        # Extract best score
        best_score = rand_clf.best_score_

        # Print best score
        logger.info("Trained with {} with score: {:.3f}".format(name, best_score))

        # Predict test set labels
        y_pred = best_model.predict(self.X_test)

        # Compute accuracy
        accuracy = accuracy_score(self.y_test, y_pred)

        # Print accuracy
        logger.info('Predicted with {} ; Test score : {:.3f}'.format(name, accuracy))
        
        return best_model, accuracy


    def train(self):
        model_params = self.config.params

        models = ConfigBox({
            "Decision_Tree": {
                "model" : DecisionTreeClassifier(),
                "params" : model_params.Decision_Tree,
                "auto":{
                        "criterion" : ['gini', 'entropy'],
                        "splitter" : ['best', 'random'],
                        "max_depth" : range( 1, 32),
                        "min_samples_split" : uniform( 0.1, 1.0),
                        "min_samples_leaf" : uniform( 0.1, 0.5),
                        "max_features" : ['auto', 'sqrt', 'log2', None],
                }
            },
            "Random_Forest": {
                "model" : RandomForestClassifier(),
                "params" : model_params.Random_Forest
                "auto":{
                    'n_estimators': range(10, 200),
                    'criterion': ['gini', 'entropy'],
                    'max_depth': range(1, 20),
                    'min_samples_split': range(2, 20),
                    'min_samples_leaf': range(1, 20),
                    'max_features': ['auto', 'sqrt', 'log2', None],
                    'bootstrap': [True, False]
                }
            },
            "SVC": {
                "model" : SVC(),
                "params" : model_params.SVC,
                "auto":{
                    'C': reciprocal(0.1, 10),  
                    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
                    'degree': range(2, 7),
                    'gamma': ['scale', 'auto'] + list(uniform(0.1, 1.0).rvs(10)),
                    'coef0': uniform(-1, 1)
                }
            },
            "LogisticRegression":{
                "model" : LogisticRegression(),
                "params" : model_params.LogisticRegression,
                "auto" : {
                        'penalty': ['l1', 'l2', 'elasticnet', 'none'],
                        'C': uniform(0.1, 10), 
                        'fit_intercept': [True, False],
                        'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
                        'max_iter': range(50, 500, 50)
                }
            },
            "MultinomialNB":{
                "model" : MultinomialNB(),
                "params" : model_params.MultinomialNB,
                "auto": {
                        'alpha': uniform(0.1, 2.0), 
                        'fit_prior': [True, False],
                        'class_prior': [None, list(uniform(0.1, 1.0).rvs(3))] 
                    }
            },
            "GradientBoost":{
                "model": GradientBoostingClassifier(),
                "params" : model_params.GradientBoost,
                "auto": {
                        'learning_rate': uniform(0.01, 0.2),  # Uniform distribution for learning_rate
                        'n_estimators': range(50, 200, 30),
                        'max_depth': range(3, 10),
                        'min_samples_split': uniform(0.1, 1.0),  # Uniform distribution for min_samples_split
                        'min_samples_leaf': uniform(0.1, 0.5),   # Uniform distribution for min_samples_leaf
                        'subsample': uniform(0.5, 1.0),           # Uniform distribution for subsample
                        'max_features': ['auto', 'sqrt', 'log2', None],
                }
            },
            "AdaBoost":{
                "model" : AdaBoostClassifier(),
                "params" : model_params.AdaBoost,
                "auto":{
                        'n_estimators': range(50, 200),
                        'learning_rate': uniform(0.01, 1.0),  # Uniform distribution for learning_rate
                        'algorithm': ['SAMME', 'SAMME.R'],
                        'base_estimator': [None, DecisionTreeClassifier(max_depth=1), DecisionTreeClassifier(max_depth=2)],
                        'loss': ['linear', 'square', 'exponential'],
                }
            },
            "XGBoost":{
                "model" : XGBClassifier(),
                "params" : model_params.XGBoost,
                "auto":{
                        'learning_rate': uniform(0.01, 0.2),
                        'n_estimators': range(50, 200),
                        'max_depth': range(3, 10),
                        'min_child_weight': range(1, 10),
                        'subsample': uniform(0.5, 1.0),
                        'colsample_bytree': uniform(0.5, 1.0),
                        'gamma': uniform(0, 1),
                        'reg_alpha': uniform(0, 1),
                        'reg_lambda': uniform(0, 1),
                        'scale_pos_weight': range(1, 10),
                        'base_score': uniform(0.1, 0.9),
                        'booster': ['gbtree', 'gblinear', 'dart'],
                        'n_jobs': [-1],
                        'random_state': range(1, 100),
                }
            },
        })

        create_directories([os.path.join(self.config.root_dir, "models")])
        trained_models = []
        for model in models:
            clf = models[model].model
            params = models[model].params

            if self.config.auto_select:
                params = models[model].auto
            else:
                params = models[model].params
            if model=="XGBoost":
                clf_model, score = self._randomized_search(name=str(model) ,clf=clf, params=params, runs=300)
            else:
                clf_model, score = self._randomized_search(name=str(model) ,clf=clf, params=params)
            trained_models.append((clf_model, score))

            save_bin(data=clf_model, path=Path(os.path.join(self.config.root_dir, f"models/{str(model)}.joblib")))
        
        trained_models = sorted(trained_models, key=lambda x:x[1], reverse=True)  # [(model, score), (model, score), ..]
        best_model = trained_models[0][0]  # taking the model

        save_bin(data=best_model, path=Path(os.path.join(self.config.root_dir, self.config.model_name)))
        save_bin_dup(data=best_model, path=Path(os.path.join(self.config.permanent_path, "model.joblib")))

        best_model_name = str(best_model)[:str(best_model).find("(")]
        best_model_score = round(trained_models[0][1], 3)
        logger.info(f"Saved main model as {best_model_name}, with score - {best_model_score}")

In [7]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2023-12-16 10:30:15,737: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-12-16 10:30:15,741: INFO: common: yaml file: params.yaml loaded successfully]
[2023-12-16 10:30:15,749: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-12-16 10:30:15,750: INFO: common: created directory at: artifacts]
[2023-12-16 10:30:15,751: INFO: common: created directory at: artifacts/model_trainer]
[2023-12-16 10:30:15,769: INFO: common: created directory at: artifacts/model_trainer/models]


/home/izam/.local/lib/python3.10/site-packages/sklearn/model_selection/_search.py:307: UserWarning: The total space of parameters 4 is smaller than n_iter=50. Running 4 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


[2023-12-16 10:30:17,814: INFO: 2321620229: Trained with Decision_Tree with score: 0.726]
[2023-12-16 10:30:17,817: INFO: 2321620229: Predicted with Decision_Tree ; Test score : 0.738]
[2023-12-16 10:30:17,820: INFO: common: binary file saved at: artifacts/model_trainer/models/Decision_Tree.joblib]


/home/izam/.local/lib/python3.10/site-packages/sklearn/model_selection/_search.py:307: UserWarning: The total space of parameters 2 is smaller than n_iter=50. Running 2 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


[2023-12-16 10:30:18,961: INFO: 2321620229: Trained with Random_Forest with score: 0.795]
[2023-12-16 10:30:18,983: INFO: 2321620229: Predicted with Random_Forest ; Test score : 0.815]
[2023-12-16 10:30:19,032: INFO: common: binary file saved at: artifacts/model_trainer/models/Random_Forest.joblib]


...

[2023-12-16 10:30:32,354: INFO: 2321620229: Trained with AdaBoost with score: 0.799]
[2023-12-16 10:30:32,406: INFO: 2321620229: Predicted with AdaBoost ; Test score : 0.814]
[2023-12-16 10:30:32,462: INFO: common: binary file saved at: artifacts/model_trainer/models/AdaBoost.joblib]


/home/izam/.local/lib/python3.10/site-packages/sklearn/model_selection/_search.py:307: UserWarning: The total space of parameters 4 is smaller than n_iter=50. Running 4 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


[2023-12-16 10:30:32,980: INFO: 2321620229: Trained with XGBoost with score: 0.799]
[2023-12-16 10:30:32,985: INFO: 2321620229: Predicted with XGBoost ; Test score : 0.820]
[2023-12-16 10:30:32,988: INFO: common: binary file saved at: artifacts/model_trainer/models/XGBoost.joblib]


/home/izam/.local/lib/python3.10/site-packages/sklearn/model_selection/_search.py:307: UserWarning: The total space of parameters 4 is smaller than n_iter=50. Running 4 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/home/izam/miniconda3/envs/customerchurn/lib/python3.10/site-packages/dask/dataframe/_pyarrow_compat.py:23: UserWarning: You are using pyarrow version 11.0.0 which is known to be insecure. See https://www.cve.org/CVERecord?id=CVE-2023-47248 for further details. Please upgrade to pyarrow>=14.0.1 or install pyarrow-hotfix to patch your current version.
  warnings.warn(
/home/izam/miniconda3/envs/customerchurn/lib/python3.10/site-packages/dask/dataframe/_pyarrow_compat.py:23: UserWarning: You are using pyarrow version 11.0.0 which is known to be insecure. See https://www.cve.org/CVERecord?id=CVE-2023-47248 for further details. Please upgrade to pyarrow>=14.0.1 or install pyarrow-hotfix to patch your current version.
  warnings.warn(
/home/izam/minicond

[2023-12-16 10:30:33,983: INFO: 2321620229: Trained with LGBM with score: 0.799]
[2023-12-16 10:30:33,987: INFO: 2321620229: Predicted with LGBM ; Test score : 0.807]
[2023-12-16 10:30:33,992: INFO: common: binary file saved at: artifacts/model_trainer/models/LGBM.joblib]
[2023-12-16 10:30:34,001: INFO: common: binary file saved at: artifacts/model_trainer/model.joblib]
[2023-12-16 10:30:34,008: INFO: common: binary file saved at: saved_models/model_01.joblib]
[2023-12-16 10:30:34,011: INFO: 2321620229: Saved main model as XGBClassifier, with score - 0.82]
